# Gray counter

There are cases where you construct a logic out of a chain of primitive elements.
Take a gray counter, for example. It is used often for cross-clock-domain negotations
(dual clock FIFO fill counters).

Note that most of the instancing below is done by logical gate connections without using a process.

In [1]:
from cyhdl import *


@block
def gray_counter1(clk : ClkSignal,
                  enable : Signal,
                  reset : ResetSignal,
                  gray_count :Signal.Output):

    n = len(gray_count)
    gray_bits = [ Signal(bool(), name="u%d" % i) for i in range(n) ]
    code, work = [ Signal(modbv()[n:]) for _ in range(2) ]
    flags = [ Signal(bool(), name="flags%d" % i) for i in range(n + 1) ]
    toggle = Signal(bool(1))
    
    grc = gray_count.clone()

    # This creates connection instances:
    connections = [
        flags[0].wireup(False),
        code.wireup(concat(*reversed(gray_bits))),
        work.wireup(concat("1", code[n-2:], toggle)),
        gray_count.wireup(grc)
    ]  
        
    for i in range(n):
        v = work[i] & ~flags[i]
        connections += [
            gray_bits[i].wireup(v ^ grc[i]),
            flags[i + 1].wireup(flags[i] | v )
        ]
                
    @always_seq(clk.posedge, reset)
    def worker():
        if enable == True:
            grc.next = code
            toggle.next = ~toggle

    return instances()

## Simulation

This particular implementation translates via the yosys' CXXRTL simulator backend, using the 'new style' factory class.

The above code is elaborated implicitely using direct RTL transfer, compiled into C++ code and executed. The result of this is a trace dump into a VCD file.

For fun, we create a `Waveform` class from an Iterator first:

In [2]:
from myirl.kernel.loops import Iterator

class Waveform(Iterator):
    def __init__(self, val, name = 'waveform'):
        it = [ True if i == '1' else False for i in val]
        super().__init__(it, name)
    def convert(self, tgt, tsz = None):
        return "True" if self.val else "False"
    def resolve_type(self):
        return bool

In [3]:
from cyhdl import *

class example_design(cyrite_factory.Module):
    # Note: in ipython, it is mandatory for a class to have an __init__ function
    # in order to retrieve the source for compilation
    def __init__(self, name, simclass, SIZE, *args):
        super().__init__(name, simclass, *args)
        self.size = SIZE

    @cyrite_factory.testbench('ns')
    def tb_gray1(self):
        SIZE = self.size
        clk = self.ClkSignal(name = 'clk')
        ce = self.Signal(bool(), name='ce')

        reset = self.ResetSignal(0, 1, isasync = True)

        dout = self.Signal(intbv()[SIZE:])    

        @self.always(delay(2))
        def clkgen():
            clk.next = ~clk

        gc = gray_counter1(clk, ce, reset, dout)

        @self.sequence
        def stim():
            ce.next = False
            reset.next = True
            yield delay(6)
            reset.next = False
            for i in Waveform("0011111111111111111111111111111111000"):
                ce.next = i       
                yield clk.negedge

            raise StopSimulation

        return instances()  
    


Run the test bench for a few cycles.

In [4]:
from yosys.simulator import CXXRTL

# Alternatively, we can fall back to GHDL:
from myirl.test.ghdl import GHDL

SIM = CXXRTL


d = example_design("myhdl_sim", SIM, 5)
t = d.tb_gray1()
t.run(180, wavetrace = t.name + ".vcd")
    

DEBUG: Dummysignal `s_c209` : False -> False
DEBUG: Dummysignal `s_c209` : False -> False
 Adding module with name `gray_counter1` 
 FINALIZE implementation `gray_counter1` of `gray_counter1` 
Compiling /tmp/myirl_myhdl_sim_fnismj40/gray_counter1.pyx because it changed.
[1/1] Cythonizing /tmp/myirl_myhdl_sim_fnismj40/gray_counter1.pyx
running build_ext
building 'runtime.gray_counter1' extension
creating build/temp.linux-x86_64-3.10/tmp/myirl_myhdl_sim_fnismj40
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=gray_counter1 -Iruntime -I/tmp/myirl_myhdl_sim_fnismj40/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_myhdl_sim_fnismj40/gray_counter1.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_myhdl_sim_fnismj40/gray_counter1.o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=gray_counter1 -Iruntime -I/tmp/myirl_myhdl_sim_fnismj40/ -I/usr/s

Using '/tmp/myirl_myhdl_sim_fnismj40/' for output
STOP SIMULATION @152


### Waveform display

The resulting waveform is displayed using a signal filter configuration:

In [5]:
waveconfig = { '.reset' : None, '.clk' : None, '.enable' : None }
for i in range(5):
    waveconfig[".u%d" % i] = None

from cyrite import waveutils
waveutils.draw_wavetrace(t, 'testbench.vcd', 'clk', cfg = waveconfig)

### Notes

This is not so much a recommended design style, as the above `gray_counter1` implementation can be owned by several contexts, as it's defined as a global function.
If several different contexts are referring to it as a top level object, unwanted effects may occur.

Therefore it is better to stick all kind of parametrizable top level objects into a design context class.

## HDL translation

To explicitely convert to HDL, we create an instance and elaborate:

In [6]:
def convert(SIZE):
    clk = ClkSignal()
    ce = Signal(bool())

    reset = ResetSignal(0, 1, isasync = True)

    dout = Signal(intbv()[SIZE:])    

    gc = gray_counter1(clk, ce, reset, dout)
    
    f = gc.elab(targets.Verilog)
    return f

f = convert(8)

 Module gray_counter1: Existing instance gray_counter1, rename to gray_counter1_1 
 Writing 'gray_counter1_1' to file /tmp/myirl_gray_counter1_vk9a_tg9/gray_counter1_1.v 
DEBUG Fallback wire for s_a905


In [7]:
!cat {f[0]}

// File generated from source:
//     /tmp/ipykernel_40607/4268148147.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture cyriteHDL

module gray_counter1_1
    (
        input wire /* std_ulogic */ clk,
        input wire /* std_ulogic */ enable,
        input wire /* std_ulogic */ reset,
        output wire [7:0] gray_count
    );
    // Local type declarations
    // Signal declarations
    reg [7:0] grc;
    reg /* std_ulogic */ toggle;
    wire [7:0] code;
    wire /* std_ulogic */ flags0;
    wire /* std_ulogic */ u7;
    wire /* std_ulogic */ u6;
    wire /* std_ulogic */ u5;
    wire /* std_ulogic */ u4;
    wire /* std_ulogic */ u3;
    wire /* std_ulogic */ u2;
    wire /* std_ulogic */ u1;
    wire /* std_ulogic */ u0;
    wire [7:0] work;
    wire /* std_ulogic */ flags1;
    wire /* std_ulogic */ flags2;
    wire /* std_ulogic */ flags3;
    wire /* std_ulogic */ flags4;
    wir